# Preprocessing des données

In [10]:
!pip install rpy2 pandas sense2vec textdistance

  Using cached textdistance-4.5.0-py3-none-any.whl (31 kB)


## Export du fichier RDS au format CSV

In [1]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

readRDS = robjects.r['readRDS']
df = readRDS('.\\Wasabi-dataset\\albums_all_artists_3000.rds')

# do something with the dataframe

Unable to determine R home: [WinError 2] Le fichier spécifié est introuvable


In [2]:
import rpy2.robjects as ro

df2 = ro.conversion.rpy2py(df)

In [3]:
df2

,_id,name,urlWikipedia,genre,length,urlAlbum,id_artist,title,publicationDate,urlAmazon,...,deezerFans,explicitLyrics,upc,id_album_musicbrainz,country,disambiguation,barcode,dateRelease,language,id_album_discogs
1,5714debf25ac0d8aee37cb42,Tommy Dorsey,,,,http://lyrics.wikia.com/Tommy_Dorsey:Other_Songs,56d98a0ccc2ddd0c0f6bdd9b,Other Songs,,,...,77,,078636635323,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,5714debf25ac0d8aee37d282,Tricky,http://en.wikipedia.org/wiki/Maxinquaye,Trip Hop,57:17,http://lyrics.wikia.com/Tricky:Maxinquaye_%281...,56d98bdecc2ddd0c0f6be067,Maxinquaye,1995,http://www.amazon.com/exec/obidos/redirect?lin...,...,7706,,731452408921,52a0845e-ca19-439e-a428-17270c0e1990,DE,,731452408921,1995-02-21,eng,50297
3,5714debf25ac0d8aee37d283,Tricky,http://en.wikipedia.org/wiki/Nearly_God,Trip Hop,64:44,http://lyrics.wikia.com/Tricky:Nearly_God_%281...,56d98bdecc2ddd0c0f6be067,Nearly God,1996,http://www.amazon.com/exec/obidos/redirect?lin...,...,646,,731452424525,46ce50f1-3838-4655-992d-a7c6a46acc29,US,,016253106420,1996-08-13,eng,979231
4,5714debf25ac0d8aee37d284,Tricky,,Trip Hop,21:16,http://lyrics.wikia.com/Tricky:Tricky_Presents...,56d98bdecc2ddd0c0f6be067,Tricky Presents Grassroots,1996,http://www.amazon.com/exec/obidos/redirect?lin...,...,2485,,731452430229,c7f08459-5c85-4a41-a760-3e6c2b6b35aa,US,,769712413026,1996,eng,202119
5,5714debf25ac0d8aee37d285,Tricky,http://en.wikipedia.org/wiki/Pre-Millennium_Te...,Trip Hop,45:37,http://lyrics.wikia.com/Tricky:Pre-Millennium_...,56d98bdecc2ddd0c0f6be067,Pre-Millennium Tension,1996,http://www.amazon.com/exec/obidos/redirect?lin...,...,2485,,731452430229,4b994ee5-38c4-4fab-bc77-83fd172a3807,US,,731452430229,1996-11-19,eng,50328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7794,5714debf25ac0d8aee37f161,Wheat,,,,http://lyrics.wikia.com/Wheat:That%27s_Exactly...,56d99384cc2ddd0c0f6bec0e,That's Exactly What I Wanted... Exactly That,2007,,...,4,,634457405394,de700a0e-ed7b-4413-b21c-3a70f8aa2d47,US,,803645002527,2006-12-15,eng,897348
7795,5714debf25ac0d8aee37f162,Wheat,,,,http://lyrics.wikia.com/Wheat:Everyday_I_Said_...,56d99384cc2ddd0c0f6bec0e,Everyday I Said A Prayer For Kathy And Made A ...,2007,,...,6,,730003501029,7318aa98-1b29-4031-9242-b5c04a129b94,US,,,2007-05-22,eng,329330
7796,5714debf25ac0d8aee37f163,Wheat,,,,http://lyrics.wikia.com/Wheat:Other_Songs,56d99384cc2ddd0c0f6bec0e,Other Songs,,,...,4,,884977881400,NULL,NULL,NULL,NULL,NULL,NULL,NULL
7797,5714debf25ac0d8aee37f164,When Death Becomes You,,,,http://lyrics.wikia.com/When_Death_Becomes_You...,56d99384cc2ddd0c0f6bec0f,Other Songs,,,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [4]:
df2.to_csv('df2.csv')

## Parsing des caractères HTML

In [5]:
import pandas as pd 
import html

df3 = pd.DataFrame({
    'name': df2['name'],
    'title': df2['title'],
    'country': [html.unescape(item) if (not item is None and not item == '' and not item == 'NULL') else 'unknown' for item in df2['country']],
    'genre': [html.unescape(item.lower()) if (not item is None and not item == '') else 'unknown' for item in df2['genre']],
    'date': [html.unescape(item) if (not item is None and not item == '') else 'unknown' for item in df2['publicationDate']]
})

df3

## Regroupement des genres en une liste plus restreinte

In [6]:
len(set(df3['genre']))

197

Ce nombre de genres est beaucoup trop important, on va regrouper les genres entre eux

Décommentez cette cellule et lancez là pour télécharger le vocabulaire Sense2Vec 2019 (>4GB)

In [7]:
#!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2019_lg.tar.gz.001
#!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2019_lg.tar.gz.002
#!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2019_lg.tar.gz.003
#!cat s2v_reddit_2019_lg.tar.gz.* > s2v_reddit_2019_lg.tar.gz
#!tar -xf s2v_reddit_2019_lg.tar.gz

In [8]:
from sense2vec import Sense2Vec

s2v = Sense2Vec().from_disk('s2v_reddit_2019_lg')

In [9]:
s2v.most_similar("apple|NOUN", n=10)

[('apple|PROPN', 0.801),
 ('apple|ORG', 0.7091),
 ('Apple|ORG', 0.6978),
 ('Apple|PROPN', 0.6511),
 ('only_apple|NOUN', 0.6467),
 ('apples|NOUN', 0.6437),
 ('other_apple|NOUN', 0.6401),
 ('blackberry|NOUN', 0.6394),
 ('apple_&amp|ORG', 0.6283),
 ('granny_smith|PROPN', 0.6276)]

In [10]:
base_genres = ['rock', 'metal', 'jazz', 'classic', 'hip hop', 'electro', 'pop', 'soul', 'folk', 'rap']

In [11]:
base_vectors = [s2v[item.replace(' ', '_') + '|NOUN'] for item in base_genres]

Ici pour chaque album, si on ne connait pas le genre alors on associe le genre au genre majoritaire de l'artiste si possible, puis on regroupe dans un genre générak de la manière suivante:

Si le nom d'un genre principal est dans le nom du genre alors on le regroupe dans le genre général

Si le nom est inclu dans le vocabulaire Sense2Vec on le regroupe au genre général le plus proche (par produit scalaire)

Sinon on l'associe au genre général avec la distance de Levenstein la plus faible

In [12]:
from scipy import spatial
import numpy as np
import textdistance
from tqdm import tqdm

df_genre = []

df_subgenres = df3['genre'][:]

for i, subgenre in tqdm(enumerate(df_subgenres)):
    if subgenre == 'unknown':
        
        related_genres = [
            df3['genre'][j] for j in range(len(df3['genre']))
            if df3['name'][j] == df3['name'][i] and i != j and df3['genre'][j] != 'unknown'
        ]
        
        genres_count = [related_genres.count(item) for item in related_genres]
        
        if len(genres_count) == 0:
            df_genre.append('unknown')
            continue
        
        df3['genre'][i] = related_genres[np.argmax(genres_count)]
        subgenre = df3['genre'][i]
        
    found = False
    found_in_subgenre = [j for j in range(len(base_genres)) if subgenre.count(base_genres[j]) > 0]
    
    if len(found_in_subgenre) > 0:
        df_genre.append(base_genres[found_in_subgenre[0]])
        continue
    
    subgenre_vector = s2v[subgenre + '|NOUN']
    
    if not subgenre_vector is None:
        dotproducts = [np.dot(subgenre_vector, genre_vector) for genre_vector in base_vectors]
        df_genre.append(base_genres[np.argmax(dotproducts)])
        continue
        
    distances = [textdistance.hamming.similarity(subgenre, genre) for genre in base_genres]
    
    df_genre.append(base_genres[np.argmax(distances)])

7798it [04:41, 27.69it/s] 


In [13]:
df4 = pd.DataFrame({
    'name': df3['name'],
    'title': df3['title'],
    'country': df3['country'],
    'genre': df_genre,
    'subgenre': df3['genre'],
    'date': df3['date']
})

In [14]:
df4[50:100]

,name,title,country,genre,subgenre,date
51,Tommy Edwards,It's All In The Game,IT,unknown,unknown,1995
52,Tommy Edwards,Other Songs,unknown,unknown,unknown,unknown
53,Tommy Emmanuel,The Journey,unknown,unknown,unknown,1993
54,Tommy Emmanuel,The Journey Continues,unknown,unknown,unknown,1993
55,Tommy Emmanuel,Can't Get Enough,unknown,unknown,unknown,1996
56,Tommy Emmanuel,Live One,unknown,unknown,unknown,2001
57,Tommy Emmanuel,Endless Road,unknown,unknown,unknown,2004
58,Tommy Emmanuel,Other Songs,unknown,unknown,unknown,unknown
59,Tommy Engel,Du Bes Kölle,unknown,unknown,unknown,2006
60,Tommy Flanagan,Other Songs,unknown,unknown,unknown,unknown


In [15]:
df4.to_csv('df4.csv')

In [16]:
len([item for item in df4['country'] if item == 'unknown']) / len(df4['country'])

0.7351885098743267

73% des ambums n'ont pas de pays... il va falloir diminuer ce nombre

Le pays d'enregistrement des albums sans pays est associé au pays majoritaire d'enregistrement de l'artiste si possible

In [17]:
#from langdetect import detect
from tqdm import tqdm

for i in tqdm(range(len(df4['title']))):
    if df4['country'][i] != 'unknown':
        continue
        
    related_samples = [j for j in range(len(df4['country'])) if df4['name'][j] == df4['name'][i] and i != j]
    
    if len(related_samples) == 0:
        continue

    related_countries = [df4['country'][j] for j in related_samples if df4['country'][j] != 'unknown']
    countries_count = [related_countries.count(item) for item in related_countries]
    
    if len(countries_count) == 0:
        continue
        
    df4['country'][i] = related_countries[np.argmax(countries_count)]

100%|██████████████████████████████████████████████████████████████████████████████| 7798/7798 [05:29<00:00, 23.66it/s]


In [18]:
len([item for item in df4['country'] if item == 'unknown']) / len(df4['country'])

0.4989740959220313

On passe de 73% à 49%... C'est mieux

Des dictionnaires pour ajouter des informations aux fichiers de sorties qui seront utiles au moment de faire la visualisation

In [19]:
country_to_continent = {
    'AR': 'America',
    'AT': 'Europe',
    'AU': 'Australia',
    'BE': 'Europe',
    'BR': 'America',
    'CA': 'America',
    'CL': 'America',
    'CZ': 'Europe',
    'DE': 'Europe',
    'DK': 'Europe',
    'EE': 'Europe',
    'ES': 'Europe',
    'FI': 'Europe',
    'FO': 'Europe',
    'FR': 'Europe',
    'GB': 'Europe',
    'GR': 'Europe',
    'HR': 'Europe',
    'IN': 'Asia',
    'IT': 'Europe',
    'JP': 'Asia',
    'MX': 'America',
    'NL': 'Europe',
    'NO': 'Europe',
    'PL': 'Europe',
    'PT': 'Europe',
    'RO': 'Europe',
    'RS': 'Europe',
    'RU': 'Asia',
    'SE': 'Europe',
    'SI': 'Europe',
    'UA': 'Europe',
    'US': 'America',
    'VE': 'America',
    'XE': 'Europe',
    'XW': 'unknown',
    'YU': 'Europe',
    'ZA': 'Africa',
    'unknown': 'unknown'
}

In [20]:
country_to_GPS = {
    'AR': { 'latitude': -23.375665, 'longitude': -68.884277 },
    'AT': { 'latitude': 47.6975425, 'longitude': 13.34931899999998 },
    'AU': { 'latitude': -26.853387500000004, 'longitude': 133.27515449999999 },
    'BE': { 'latitude': 50.503906, 'longitude': 4.476982 },
    'BR': { 'latitude': -14.235004, 'longitude': -51.92528 },
    'CA': { 'latitude': 62.393303, 'longitude': -96.8181455 },
    'CL': { 'latitude': -36.7119505, 'longitude': -73.601704 },
    'CZ': { 'latitude': 50.0755381, 'longitude': 4.476982 },
    'DE': { 'latitude': 51.165691, 'longitude': 10.451526 },
    'DK': { 'latitude': 56.15540300000001, 'longitude': 11.617222500000025 },
    'EE': { 'latitude': 58.5962085, 'longitude': 25.023777999999993 },
    'ES': { 'latitude': 39.896026500000005, 'longitude': -2.487694499999975 },
    'FI': { 'latitude': 64.9524155, 'longitude': 26.068943999999988 },
    'FO': { 'latitude': 61.892635, 'longitude': -6.911806},
    'FR': { 'latitude': 46.232192999999995, 'longitude': 2.209666999999996 },
    'GB': { 'latitude': 52.3555177, 'longitude': -1.1743197 },
    'GR': { 'latitude': 39.074208, 'longitude': 21.824312 },
    'HR': { 'latitude': 45.100000, 'longitude': 15.200000 },
    'IN': { 'latitude': 20.595164, 'longitude': 78.963606 },
    'IT': { 'latitude': 41.8739875, 'longitude': 12.564166999999998 },
    'JP': { 'latitude': 34.886306, 'longitude': 134.379711 },
    'MX': { 'latitude': 23.634501, 'longitude': -102.552784 },
    'NL': { 'latitude': 52.132633, 'longitude': 5.291266 },
    'NO': { 'latitude': 60.472024, 'longitude': 8.468946 },
    'PL': { 'latitude': 51.9227505, 'longitude': 19.136874499999976 },
    'PT': { 'latitude': 39.553444, 'longitude': -7.839318999999989 },
    'RO': { 'latitude': 45.9471245, 'longitude': 24.9805135 },
    'RS': { 'latitude': 44.206802368164, 'longitude': 20.91100883483898 },
    'RU': { 'latitude': 61.5231115, 'longitude': 105.10000000000002 },
    'SE': { 'latitude': 62.199805999999995, 'longitude': 17.63749324195902 },
    'SI': { 'latitude': 46.1455285, 'longitude': 14.974541499999987 },
    'UA': { 'latitude': 48.3798885, 'longitude': 31.168139499999995 },
    'US': { 'latitude': 36.966428, 'longitude': -95.844032 },
    'VE': { 'latitude': 6.423750, 'longitude': -66.589730 },
    'XE': { 'latitude': 54.525961, 'longitude': 15.255119 },
    'XW': { 'latitude': 0, 'longitude': 0},
    'YU': { 'latitude': 44.206802368164, 'longitude': 20.91100883483898 },
    'ZA': { 'latitude': -28.48322, 'longitude': 24.676997 },
    'unknown': { 'latitude': 0, 'longitude': 0}
}

In [21]:
country_full_name = {
    'AR': 'Argentina',
    'AT': 'Austria',
    'AU': 'Australia',
    'BE': 'Belgium',
    'BR': 'Brazil',
    'CA': 'Canada',
    'CL': 'Chile',
    'CZ': 'Czech Republic',
    'DE': 'Germany',
    'DK': 'Denmark',
    'EE': 'Estonia',
    'ES': 'Spain',
    'FI': 'Finland',
    'FO': 'Faroe Islands',
    'FR': 'France',
    'GB': 'Great Britain',
    'GR': 'Greece',
    'HR': 'Croatia',
    'IN': 'India',
    'IT': 'Italy',
    'JP': 'Japan',
    'MX': 'Mexico',
    'NL': 'Netherlands',
    'NO': 'Norway',
    'PL': 'Poland',
    'PT': 'Portugal',
    'RO': 'Romania',
    'RS': 'Serbia',
    'RU': 'Russia',
    'SE': 'Sweden',
    'SI': 'Slovenia',
    'UA': 'Ukraine',
    'US': 'United States of America',
    'VE': 'Venezuela',
    'XE': 'European',
    'XW': 'World wide',
    'YU': 'Yugoslavia',
    'ZA': 'South Africa',
    'unknown': 'Unknown'
}

In [22]:
continent_to_GPS = {
    'Europe': { 'latitude': 54.525961, 'longitude': 15.255119 },
    'America': { 'latitude': 9.752562, 'longitude': -83.838431 },
    'Africa': { 'latitude': 4.070479, 'longitude': 21.187447 },
    'Asia': { 'latitude': 52.333207, 'longitude': 84.041506 },
    'Australia': {'latitude': -24.54575, 'longitude': 131.561479},
    'unknown': {'latitude': 0, 'longitude': 0}
}

In [23]:
continent_to_code = {
    'Europe': 'EUR',
    'America': 'AM',
    'Africa': 'AFC',
    'Asia': 'ASA',
    'Australia': 'OCE',
    'unknown': 'unknown'
}

In [24]:
continents = [country_to_continent[item] for item in df4['country']]

In [25]:
df5 = pd.DataFrame({
    'name': df4['name'],
    'title': df4['title'],
    'country': df4['country'],
    'genre': df_genre,
    'subgenre': df4['subgenre'],
    'date': df4['date'],
    'continent': continents
})

In [26]:
df5[100:150]

,name,title,country,genre,subgenre,date,continent
101,Trife Diesel,Better Late Than Never,US,hip hop,hip hop,2008,America
102,Trife Diesel,Other Songs,US,hip hop,hip hop,unknown,America
103,Trill Fam,Other Songs,unknown,unknown,unknown,unknown,unknown
104,Trikot,Sonntag,unknown,pop,pop,2012,unknown
105,Trikot,Other Songs,unknown,pop,pop,unknown,unknown
106,Trillium,Other Songs,unknown,unknown,unknown,unknown,unknown
107,Trillit,Other Songs,unknown,unknown,unknown,unknown,unknown
108,Trilogy,Other Songs,unknown,unknown,unknown,unknown,unknown
109,Trilobite,Other Songs,unknown,unknown,unknown,unknown,unknown
110,Trillville,The King of Crunk & BME Recordings Present: Tr...,unknown,unknown,unknown,2004,unknown


In [27]:
df5.to_csv('df5.csv')

In [28]:
import pandas as pd

df5 = pd.read_csv('df5.csv')

In [29]:
years = [int(df5['date'][i]) for i in range(len(df5)) if df5['date'][i].isdigit()]
print(min(years), max(years))

1955 2016


## Segmentation des données par tranches de 15 ans

In [30]:
years1950 = [i for i in range(len(df5)) if df5['date'][i].isdigit() and int(df5['date'][i]) >= 1950 and int(df5['date'][i]) < 1965]
years1965 = [i for i in range(len(df5)) if df5['date'][i].isdigit() and int(df5['date'][i]) >= 1965 and int(df5['date'][i]) < 1980]
years1980 = [i for i in range(len(df5)) if df5['date'][i].isdigit() and int(df5['date'][i]) >= 1980 and int(df5['date'][i]) < 1995]
years1995 = [i for i in range(len(df5)) if df5['date'][i].isdigit() and int(df5['date'][i]) >= 1995 and int(df5['date'][i]) < 2010]
years2010 = [i for i in range(len(df5)) if df5['date'][i].isdigit() and int(df5['date'][i]) >= 2010 and int(df5['date'][i]) < 2025]
yearsunknown =  [i for i in range(len(df5)) if not df5['date'][i].isdigit()]

In [31]:
columns = list(df5.columns)[1:]

In [32]:
columns

['name', 'title', 'country', 'genre', 'subgenre', 'date', 'continent']

In [33]:
years1950 = pd.DataFrame({column: [df5[column][i] for i in years1950] for column in columns})
years1965 = pd.DataFrame({column: [df5[column][i] for i in years1965] for column in columns})
years1980 = pd.DataFrame({column: [df5[column][i] for i in years1980] for column in columns})
years1995 = pd.DataFrame({column: [df5[column][i] for i in years1995] for column in columns})
years2010 = pd.DataFrame({column: [df5[column][i] for i in years2010] for column in columns})
yearsunknown = pd.DataFrame({column: [df5[column][i] for i in yearsunknown] for column in columns})

In [34]:
#years1950

In [35]:
len([item for item in df5['country'] if item == 'unknown']) / len(df5['country'])

0.4989740959220313

Les 3 fonctions make_flare servent à créert différentes hiérarchies selon ce que l'on souhaite

Le json est un arbre d'éléments dont certains ont des enfants via leur champs children

La 1ère version crée une hiéarchie continents > pays > artistes

La 2ème version fait une hiérarchie continents > pays > genres

La dernière version faire une hiérarchie continent > pays

In [36]:
from tqdm import tqdm
# flare json format

def make_flare(df):
    flare = {"name": "World"}

    continents = list(set(df5['continent']))

    continents_data = []
    for continent in continents:
        continent_data = {'name': continent}
        continent_samples = [i for i in range(len(df5)) if df5['continent'][i] == continent]
        countries = list(set([df5['country'][i] for i in continent_samples]))

        countries_data = []
        for country in tqdm(countries):
            country_data = {'name': country_full_name[country]}
            country_samples = [i for i in continent_samples if df5['country'][i] == country]
            artists = list(set([df5['name'][i] for i in country_samples]))

            artists_data = []
            for artist in artists:
                artist_data = {'name': artist}
                artist_samples = [i for i in country_samples if df5['name'][i] == artist]
                artist_data['size'] = len(artist_samples)
                artists_data.append(artist_data)
            country_data['children'] = artists_data
            countries_data.append(country_data)
        continent_data['children'] = countries_data
        continents_data.append(continent_data)
    flare['children'] = continents_data
    return flare

In [37]:
from tqdm import tqdm
# flare json format

def make_flare(df):
    flare = {"name": "World"}

    continents = list(set(df['continent']))

    continents_data = []
    for continent in continents:
        continent_data = {'name': continent}
        continent_samples = [i for i in range(len(df)) if df['continent'][i] == continent]
        countries = list(set([df['country'][i] for i in continent_samples]))

        countries_data = []
        for country in tqdm(countries):
            country_data = {'name': country_full_name[country]}
            country_samples = [i for i in continent_samples if df['country'][i] == country]
            genres = list(set([df['genre'][i] for i in country_samples]))

            genres_data = []
            for genre in genres:
                genre_data = {'name': genre}
                genre_samples = [i for i in country_samples if df['genre'][i] == genre]
                genre_data['size'] = len(genre_samples)
                genres_data.append(genre_data)
            country_data['children'] = genres_data
            countries_data.append(country_data)
        continent_data['children'] = countries_data
        continents_data.append(continent_data)
    flare['children'] = continents_data
    return flare

In [38]:
from tqdm import tqdm
# flare json format

def make_flare(df):
    flare = {"name": "World"}

    continents = list(set(df5['continent']))

    continents_data = []
    for continent in continents:
        continent_data = {'name': continent}
        continent_samples = [i for i in range(len(df5)) if df5['continent'][i] == continent]
        countries = list(set([df5['country'][i] for i in continent_samples]))

        countries_data = []
        for country in tqdm(countries):
            country_data = {'name': country_full_name[country]}
            country_samples = [i for i in continent_samples if df5['country'][i] == country]
            country_data['size'] = len(country_samples)
            countries_data.append(country_data)
            
        continent_data['children'] = countries_data
        continents_data.append(continent_data)
    flare['children'] = continents_data
    return flare

In [63]:
jsonBig = make_flare(df5)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.70it/s]


In [64]:
with open('wazajson-v8.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(jsonBig))

In [12]:
json1950 = make_flare(years1950)
json1965 = make_flare(years1965)
json1980 = make_flare(years1980)
json1995 = make_flare(years1995)
json2010 = make_flare(years2010)
jsonunknown = make_flare(yearsunknown)

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2914.74it/s]


In [13]:
import json

with open('json1950.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(json1950))
    
with open('json1965.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(json1965))
    
with open('json1980.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(json1980))
               
with open('json1995.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(json1995))
               
with open('json2010.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(json2010))
               
with open('jsonunknown.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(jsonunknown))

In [35]:
import json

with open('wazajson-v6.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(flare))

In [36]:
[item['name'] for item in flare['children']]

['unknown', 'America', 'Asia', 'Africa', 'Europe', 'Australia']

Ici on a une autre version du dataset sous forme de liste. Lorsque l'on a un continent donc sans élément parent on a un champ parent à None. Lorsque l'on a un pays on renseigne son continent dans le champ "parent"

In [28]:
from tqdm import tqdm

final_json = []

continents = continent_to_code.keys()

for continent in tqdm(continents):
    continent_data = {
        'code': continent_to_code[continent],
        'parent': None,
        'name': continent
    }
    
    related_samples = [j for j in range(len(df5['continent'])) if df5['continent'][j] == continent]
    continent_data['population'] = len(related_samples)
    final_json.append(continent_data)

countries = country_full_name.keys()

for country in tqdm(countries):
    country_data = {
        'code': country,
        'parent': country_to_continent[country],
        'name': country_full_name[country]
    }
    
    related_samples = [j for j in range(len(df5['country'])) if df5['country'][j] == country]
    country_data['population'] = len(related_samples)
    final_json.append(country_data)

100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 40.02it/s]


Ici on essaie d'avoir la même structure sauf que l'on ajoute les artistes locaux au sein des pays

In [29]:
for zone in tqdm(final_json):
    if zone['parent'] is None:
        continue
    
    zone_artists = {}
        
    related_samples = [j for j in range(len(df5['country'])) if df5['country'][j] == zone['code']]
    artists = set([df5['name'][j] for j in related_samples])
    
    for artist in artists:
        albums = [j for j in related_samples if df5['name'][j] == artist]
        artist_albums = []
        
        for album in albums:
            artist_albums.append({
                'name': df5['name'][album],
                'title': df5['title'][album],
                'country': df5['country'][album],
                'genre': df5['genre'][album],
                'subgenre': df5['subgenre'][album],
                'date': df5['date'][album],
                'continent': df5['continent'][album]
            })
        
        zone_artists[artist] = artist_albums
    zone['artists'] = zone_artists

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:31<00:00,  1.42it/s]


In [30]:
import json

output = json.dumps(final_json)
with open('wazajson-v5.json', 'w') as file:
    file.write(output)

Ici on a une structure en arbre mais en ajoutant les informations des coordonnées GPS des pays et continents grâce aux dictionnaires plus haut

In [67]:
final_json = {}

continents = set(df5['continent'])

for continent in continents:
    continent_json = {
        'latitude': continent_to_GPS[continent]['latitude'],
        'longitude': continent_to_GPS[continent]['longitude'],
    }
    continent_samples = [j for j in range(len(df5['continent'])) if df5['continent'][j] == continent]
    
    countries = set([df5['country'][item] for item in continent_samples])
    
    for country in tqdm(countries):
        country_json = {
            'name': country_full_name[country],
            'latitude': country_to_GPS[country]['latitude'],
            'longitude': country_to_GPS[country]['longitude'],
            'artists': {}
        }
        
        country_samples = [j for j in continent_samples if df5['country'][j] == country]
        
        for sample in country_samples:
            artist = df5['name'][sample]
            album = {
                        'name': df5['name'][sample],
                        'title': df5['title'][sample],
                        'country': df5['country'][sample],
                        'genre': df5['genre'][sample],
                        'subgenre': df5['subgenre'][sample],
                        'date': df5['date'][sample],
                        'continent': df5['continent'][sample]
                    }
            
            if not artist in country_json['artists']:
                country_json['artists'][artist] = [album]
            else:
                country_json['artists'][artist].append(album)
        
        continent_json[country] = country_json
    final_json[continent] = continent_json

100%|█████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 128.49it/s]


In [68]:
import json

output = json.dumps(final_json)
with open('wazajson-v3.json', 'w') as file:
    file.write(output)

In [1]:
import json

In [2]:
data = None
with open('wazajson-v3.json', 'r', encoding='utf-8') as file:
    data = json.load(file)